<a href="https://colab.research.google.com/github/badadumTss/convDAE/blob/main/ConvDAE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports, background
As backend for this project we'll use [tensorflow.keras](https://keras.io/), Google's deep learning API. Along with that numpy is very useful to apply all sorts of transformations to the tensors and vector rapresenting our data (e.g. adding noise to an image).

In [1]:
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Conv2DTranspose
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.regularizers import L1
from tensorflow.keras import backend as K
import tensorflow.keras
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
matplotlib.use("Agg") # set the matplotlib backend to save images asyncronously
import cv2

# The dataset, CIFAR10
The choosen dataset for this project is the CIFAR10 dataset. It is a collection of 60000 32x32 colour images grouped in 10 classes, with 6000 images per class (source: [CIFAR10 site](https://www.cs.toronto.edu/~kriz/cifar.html)). This dataset is very suited for the project because the images are not particularly big (32x32 is easily rapresentable with a vector) and has a lot of samples to choose from. It also is one of the default datasets included in the TensorFlow library, so download it is relatively easy.

In [2]:
print("[INFO] loading CIFAR10 dataset...")
((trainX, _), (testX, _)) = cifar10.load_data()

trainX = trainX.astype("float32") / 255.0
testX = testX.astype("float32") / 255.0

def add_noise_and_clip_data(data):
   noise = np.random.normal(loc=0.0, scale=0.1, size=data.shape)
   data = data + noise
   data = np.clip(data, 0., 1.)
   return data

trainXNoisy = add_noise_and_clip_data(trainX)
testXNoisy = add_noise_and_clip_data(testX)

[INFO] loading CIFAR10 dataset...


# The LinearAutoencoder

In [3]:
class LinearAutoencoder:
  @staticmethod
  def build(width, height, depth, filters=(32,64), latentDim=16):
    inputShape = height * width * depth
    channelDim = -1
    inputs = Input(shape=(inputShape,))
    x = inputs
    for f in reversed(filters):
      x = Dense(units=f)(x)
      x = BatchNormalization()(x)
      x = LeakyReLU()(x)
    # encoded space
    latent = Dense(units=latentDim,
              activation='linear',
              activity_regularizer=L1(0.0001))(x)
    # encoder
    encoder = Model(inputs, latent, name='encoder')
    # decoder start
    latentInputs = Input(shape=(latentDim,))
    y = latentInputs
    for f in filters:
      y = Dense(units=f)(y)
      y = BatchNormalization()(y)
      y = LeakyReLU()(y)
    
    outputs = Dense(units=inputShape, activation='sigmoid')(y)
    decoder = Model(latentInputs, outputs, name='decoder')
    autoencoder = Model(inputs, outputs, decoder(encoder(inputs)), name='autoencoder')
    return (encoder, decoder, autoencoder)

# The ConvAutoencoder


In [4]:
class ConvAutoencoder:
	@staticmethod
	def build(width, height, depth, filters=(32, 64), latentDim=16):
		# initialize the input shape to be "channels last" along with
		# the channels dimension itself
		# channels dimension itself
		inputShape = (height, width, depth)
		chanDim = -1
		# define the input to the encoder
		inputs = Input(shape=inputShape)
		x = inputs
		# loop over the number of filters
		for f in filters:
			# apply a CONV => RELU => BN operation
			x = Conv2D(f, (3, 3), strides=2, padding="same")(x)
			x = LeakyReLU(alpha=0.2)(x)
			x = BatchNormalization(axis=chanDim)(x)
		# flatten the network and then construct our latent vector
		volumeSize = K.int_shape(x)
		x = Flatten()(x)
		latent = Dense(latentDim)(x)
		# build the encoder model
		encoder = Model(inputs, latent, name="encoder")
		# start building the decoder model which will accept the
		# output of the encoder as its inputs
		latentInputs = Input(shape=(latentDim,))
		x = Dense(np.prod(volumeSize[1:]))(latentInputs)
		x = Reshape((volumeSize[1], volumeSize[2], volumeSize[3]))(x)
		# loop over our number of filters again, but this time in
		# reverse order
		for f in filters[::-1]:
			# apply a CONV_TRANSPOSE => RELU => BN operation
			x = Conv2DTranspose(f, (3, 3), strides=2,
				padding="same")(x)
			x = LeakyReLU(alpha=0.2)(x)
			x = BatchNormalization(axis=chanDim)(x)
		# apply a single CONV_TRANSPOSE layer used to recover the
		# original depth of the image
		x = Conv2DTranspose(depth, (3, 3), padding="same")(x)
		outputs = Activation("sigmoid")(x)
		# build the decoder model
		decoder = Model(latentInputs, outputs, name="decoder")
		# our autoencoder is the encoder + decoder
		autoencoder = Model(inputs, decoder(encoder(inputs)),
			name="autoencoder")
		# return a 3-tuple of the encoder, decoder, and autoencoder
		return (encoder, decoder, autoencoder)

## The test function

In order to use wandb we also have to login, the given key is my personal test key, feel free to use it for now

In [5]:
!pip install wandb -qqq
import wandb

wandb.login()

wandb: Currently logged in as: badadumtss. Use `wandb login --relogin` to force relogin


True

### Configure wandb
In this section we'll configure sweep, one of wandb products that allows for optimal hyperparameter search.

In [6]:
sweep_config = {
    'method': 'random',
    'metric': {
        'name': 'loss',
        'goal': 'minimize'
    }
}

params = {
    'model': {
        'values': ['linear', 'convolutional']
    },
    'epochs': {
        'value': 5
    },
    'batch_size': {
        'value': 32
    },
    'latent_dim': {
        'values': [128, 32, 16]
    },
    'filters': {
        'values': [(32, 64), (32, 64, 128)]
    },
    'learning_rate': {
        'values': [1e-4, 1e-3, 0.01]
    }
}

sweep_config['parameters'] = params


## Initialize the sweep

In [7]:
sweep_id = wandb.sweep(sweep_config, project="convDAE")

Create sweep with ID: 0vxf9yf9
Sweep URL: https://wandb.ai/badadumtss/convDAE/sweeps/0vxf9yf9


# Define the training


In [8]:
def build_model(config):
  if config.model == 'linear':
    return LinearAutoencoder.build(32, 32, 3, filters=config.filters, latentDim=config.latent_dim)
  if config.model == 'convolutional':
    return ConvAutoencoder.build(32, 32, 3, filters=config.filters, latentDim=config.latent_dim)

In [9]:
def train(config=None):
  with wandb.init(config=config):
    config = wandb.config
    optimizer = Adam(learning_rate = config.learning_rate)
    # construct our convolutional autoencoder
    print("[INFO] building autoencoder...")
    (encoder, decoder, autoencoder) = build_model(config)

    global trainXNoisy
    global trainX
    if config.model == 'linear':
      trainXNoisy = trainXNoisy.reshape(60000, 3072)
      trainX = trainX.reshape(60000, 3072)
    
    autoencoder.compile(loss="mse", optimizer=optimizer, metrics=['accuracy'])
    # train the convolutional autoencoder
    H = autoencoder.fit(
	    trainXNoisy, trainX,
	    validation_data=(testXNoisy, testX),
	    epochs=config.epochs,
	    batch_size=config.batch_size
    )
    
    wandb.log({'epochs': config.epochs,
                'loss': np.mean(H.history['loss']),
                'val_loss': np.mean(H.history['val_loss'])})
    del encoder
    del decoder
    del autoencoder
    

# Run the sweeper

In [10]:
wandb.agent(sweep_id, train, count=5)

wandb: Agent Starting Run: ctgbmdxp with config:
wandb: 	batch_size: 32
wandb: 	epochs: 5
wandb: 	filters: [32, 64]
wandb: 	latent_dim: 16
wandb: 	learning_rate: 0.001
wandb: 	model: convolutional


[INFO] building autoencoder...
Epoch 1/5
1233/1563 [======================>.......] - ETA: 34s - loss: 0.0205 - accuracy: 0.5564

wandb: Ctrl + C detected. Stopping sweep.
